In [1]:
import torch
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from mqbench.convert_deploy import convert_deploy
from mqbench.prepare_by_platform import prepare_by_platform, BackendType
from mqbench.utils.state import enable_calibration, enable_quantization

In [2]:
import mqbench
from mqbench.convert_deploy import convert_deploy

In [3]:
import torch.nn.functional as F

In [4]:
import torch.nn as nn

class NetBN(nn.Module):

    def __init__(self, num_channels=1):
        super(NetBN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 40, 3, 1)
        self.bn1 = nn.BatchNorm2d(40)
        self.conv2 = nn.Conv2d(40, 40, 3, 1)
        self.bn2 = nn.BatchNorm2d(40)
        
        self.conv3 = nn.Conv2d(40, 40, 3, 1)
        self.bn3 = nn.BatchNorm2d(40)
        
        self.fc = nn.Linear(40, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        
        x = x.mean(dim=[2,3])
        x = self.fc(x)
        return x


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import os
import os.path as osp


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    lossLayer = torch.nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = lossLayer(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), loss.item()
            ))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    lossLayer = torch.nn.CrossEntropyLoss(reduction='sum')
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += lossLayer(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss, 100. * correct / len(test_loader.dataset)
    ))


batch_size = 256
test_batch_size = 256
seed = 1
epochs = 15
lr = 0.003
save_model = True
using_bn = True

torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=True, download=True, 
                   transform=transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                   ])),
    batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])),
    batch_size=test_batch_size, shuffle=False, num_workers=1, pin_memory=True
)

if using_bn:
    model = NetBN(num_channels=3).to(device)
else:
    model = Net(num_channels=3).to(device)

optimizer = optim.AdamW(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

if save_model:
    os.makedirs('ckpt',exist_ok = True)
    if using_bn:
        torch.save(model.state_dict(), 'ckpt/mnist_cnnbn.pt')
    else:
        torch.save(model.state_dict(), 'ckpt/mnist_cnn.pt')

Files already downloaded and verified


In [6]:
model = model.cpu().train()

if save_model:
    if using_bn:
        model.load_state_dict(torch.load('ckpt/mnist_cnnbn.pt', map_location='cpu'))
    else:
        model.load_state_dict(torch.load('ckpt/mnist_cnn.pt', map_location='cpu'))
    
model_mqbench = prepare_by_platform(model, BackendType.OPENVINO)
model_mqbench = model_mqbench.to(device)
# before training, we recommend to enable observers for calibration in several batches, and then enable quantization.
enable_calibration(model_mqbench)
model_mqbench = model_mqbench.to(device)
model_mqbench.eval()
with torch.no_grad():
    for i, (data, target) in enumerate(test_loader):
        # do forward procedures
        data = data.to(device)
        model_mqbench(data)

[MQBENCH] INFO: Quantize model Scheme: BackendType.OPENVINO Mode: Training
[MQBENCH] INFO: Weight Qconfig:
    FakeQuantize: LearnableFakeQuantize Params: {}
    Oberver:      MinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: True / Pot scale: False
[MQBENCH] INFO: Activation Qconfig:
    FakeQuantize: LearnableFakeQuantize Params: {}
    Oberver:      EMAMinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: False / Pot scale: False
[MQBENCH] INFO: Replace module to qat module.
[MQBENCH] INFO: Now all weight quantizers will effectively use only 7 bits out of 8 bits. This resolves the overflow issue problem on AVX2 and AVX-512 machines.
[MQBENCH] INFO: Insert act quant x_post_act_fake_quantizer
[MQBENCH] INFO: Set conv1 post act quantize to 8 bit unsigned type.
[MQBENCH] INFO: Insert act quant conv1_post_act_fake_quantizer
[MQBENCH] INFO: Set conv2 post act quantize to 8 bit unsigned type.
[MQBENCH] INFO: Insert act quant conv2_post_act_fake_quantizer
[MQ

In [7]:
test(model_mqbench, device, test_loader)


Test set: Average loss: 0.7246, Accuracy: 75%



In [8]:
model_mqbench.zero_grad()
model_mqbench.train()
enable_quantization(model_mqbench)

optimizer = optim.AdamW(model_mqbench.parameters(), lr=lr*0.5)
epochs = 6
for epoch in range(1, epochs + 1):
    train(model_mqbench, device, train_loader, optimizer, epoch)
    test(model_mqbench, device, test_loader)

[MQBENCH] INFO: Disable observer and Enable quantize.
Train Epoch: 1 [0/50000]	Loss: 0.956195
Train Epoch: 1 [12800/50000]	Loss: 0.753290
Train Epoch: 1 [25600/50000]	Loss: 0.719596
Train Epoch: 1 [38400/50000]	Loss: 0.769043

Test set: Average loss: 0.6960, Accuracy: 76%

Train Epoch: 2 [0/50000]	Loss: 0.625557
Train Epoch: 2 [12800/50000]	Loss: 0.648791
Train Epoch: 2 [25600/50000]	Loss: 0.678966
Train Epoch: 2 [38400/50000]	Loss: 0.708362

Test set: Average loss: 0.7010, Accuracy: 76%

Train Epoch: 3 [0/50000]	Loss: 0.683164
Train Epoch: 3 [12800/50000]	Loss: 0.770475
Train Epoch: 3 [25600/50000]	Loss: 0.796394
Train Epoch: 3 [38400/50000]	Loss: 0.800356

Test set: Average loss: 0.6729, Accuracy: 77%

Train Epoch: 4 [0/50000]	Loss: 0.646614
Train Epoch: 4 [12800/50000]	Loss: 0.622016
Train Epoch: 4 [25600/50000]	Loss: 0.771547
Train Epoch: 4 [38400/50000]	Loss: 0.644924

Test set: Average loss: 0.6832, Accuracy: 77%

Train Epoch: 5 [0/50000]	Loss: 0.777149
Train Epoch: 5 [12800/5000

In [9]:
input_shape={'data': [1, 3, 32, 32]}
convert_deploy(model_mqbench.eval(), BackendType.OPENVINO, input_shape)

[MQBENCH] INFO: Merge BN for deploy.
[MQBENCH] INFO: Export to onnx.


/home/hbtian/MQBench0130/mqbench/fake_quantize/lsq.py:62: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.observer_enabled[0] == 1:
/home/hbtian/MQBench0130/mqbench/fake_quantize/lsq.py:76: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  self.scale.data.clamp_(min=self.eps.item())
/home/hbtian/MQBench0130/mqbench/fake_quantize/lsq.py:78: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means tha

[MQBENCH] INFO: Extract qparams for OPENVINO.
[MQBENCH] INFO: Finish deploy process.


# test openvino inference

In [1]:
!pip install openvino-dev

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [10]:
!mo --input_model mqbench_qmodel_deploy_model.onnx 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/hbtian/MQBench0130/haibin_test/mqbench_qmodel_deploy_model.onnx
	- Path for generated IR: 	/home/hbtian/MQBench0130/haibin_test/.
	- IR output name: 	mqbench_qmodel_deploy_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
	- Inference Engine found in: 	/home/hbtian/miniconda3/envs/mqbench/lib/python3.7/site-packages/openvino
Inference Engine version: 	2021.4.2-3976-0943ed67223-refs/pull/539/head
Model Optimizer ve

In [11]:
import logging as log
import numpy as np
from openvino.inference_engine import IECore

In [12]:
log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.ERROR, stream=sys.stdout)
log.info("Creating Inference Engine...")
ie = IECore()

In [13]:
# Read IR
log.info("Loading network")
net = ie.read_network("./mqbench_qmodel_deploy_model.xml")

img_info_input_blob = None
feed_dict = {}
input_blob = "input"
for blob_name in net.input_info:
    if len(net.input_info[blob_name].input_data.shape) == 4:
        input_blob = blob_name
    elif len(net.input_info[blob_name].input_data.shape) == 2:
        img_info_input_blob = blob_name
    else:
        raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                           .format(len(net.input_info[blob_name].input_data.shape), blob_name))

log.info("Loading IR to the plugin...")

In [14]:
exec_net = ie.load_network(network=net, num_requests=4, device_name="CPU")
n, c, h, w = net.input_info[input_blob].input_data.shape
if img_info_input_blob:
    feed_dict[img_info_input_blob] = [h, w, 1]

In [15]:
n, c, h, w

(1, 3, 32, 32)

In [16]:
def test_openvino(exec_net, test_loader):
    
    global feed_dict
    global input_blob
    cur_request_id = 0
    
    test_loss = 0
    correct = 0
    
    def infer(img):
        cur_request_id = 0
        feed_dict[input_blob] = img
        exec_net.start_async(request_id=cur_request_id, inputs=feed_dict)
        while exec_net.requests[cur_request_id].wait(-1) != 0:
            pass
        outs = exec_net.requests[cur_request_id].output_blobs
        k = tuple(outs.keys())[0]
        return outs[k].buffer
    
    lossLayer = torch.nn.CrossEntropyLoss(reduction='sum')
    
    for data, target in test_loader:
        for subdata, sub_target in zip(data, target):
            output = infer(subdata.unsqueeze(dim=0).numpy())
            output = torch.as_tensor(output)
            test_loss += lossLayer(output, sub_target.unsqueeze(dim=0)).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(sub_target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss, 100. * correct / len(test_loader.dataset)
    ))

In [17]:
test_openvino(exec_net, test_loader)


Test set: Average loss: 0.6873, Accuracy: 77%

